Import

In [32]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup


In [33]:
soup = ""
with open("data.xml", "r", encoding="utf-8") as file:
    content = file.read()
    soup = BeautifulSoup(content, "xml")
    file.close()


#ProductTree

In [98]:
library = soup.find('ProductTree')
productdb = {'ProductID':[], 'ProductName':[], 'Type':[], 'Name':[]}


def process_node(node, productdb):

    if node.name is not None:
        if 'ProductID' in node.attrs:
            productdb['ProductID'].append(node.attrs['ProductID'])
            productdb['ProductName'].append(node.text)
        
        if 'Type' not in node.attrs and node.name != 'ProductTree':
            if 'Type' in node.parent.attrs:
                 productdb['Type'].append(node.parent.attrs['Type'])
                 productdb['Name'].append(node.parent.attrs['Name'])
            else:
                productdb['Type'].append(None)
                productdb['Name'].append(None)

        for child in node.children:
            process_node(child, productdb)


process_node(library, productdb)
productdb = pd.DataFrame(productdb)
productdb.to_excel('data.xlsx', index=False)
print(productdb)

    ProductID                                        ProductName  \
0       11568         Windows 10 Version 1809 for 32-bit Systems   
1       11569      Windows 10 Version 1809 for x64-based Systems   
2       11570    Windows 10 Version 1809 for ARM64-based Systems   
3       11571                                Windows Server 2019   
4       11572     Windows Server 2019 (Server Core installation)   
..        ...                                                ...   
177     12139                                CBL Mariner 2.0 x64   
178     12140                                CBL Mariner 2.0 ARM   
179      9312  Windows Server 2008 for 32-bit Systems Service...   
180      9318  Windows Server 2008 for x64-based Systems Serv...   
181      9344  Windows Server 2008 for x64-based Systems Serv...   

               Type     Name  
0    Product Family  Windows  
1    Product Family  Windows  
2    Product Family  Windows  
3    Product Family  Windows  
4    Product Family  Windows